# What is function introspection?

- Generally, it means analyzing your code using code

- **Recall**: functions are first-class objects
    - They contain attributes like:
        - `__doc__`
        - `__annotations__`

- We're also able to create whatever custom attributes we want

In [1]:
def my_func(a, b):
    return a + b

In [2]:
my_func.category = 'math'
my_func.sub_category = 'arithmetic'

- Now, we can access these attributes

In [3]:
my_func.category, my_func.sub_category

('math', 'arithmetic')

# How can we see a summary of all the available attributes?

- We can use the built-in function `dir`

In [4]:
dir(my_func)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'category',
 'sub_category']

# What are the most commonly used methods?

### 1. `__name__`

- Simply the name of the function
    - *Why would we need to know this?*

In [7]:
def f1(x):
    return x+1

def f2(x):
    return x+2

list_functions = [f1, f2]

In [8]:
for f in list_functions:
    print(f.__name__)

f1
f2


### 2. `__defaults__`

- Returns the default values for positional arguments

### 3. `__kwdefaults__`

- Returns the default values for keyword-only arguments

In [9]:
def my_func(a, b=2, c=3, *, kw1, kw2=2):
    pass

In [10]:
my_func.__defaults__

(2, 3)

In [11]:
my_func.__kwdefaults__

{'kw2': 2}

### 4. `__code__`

- If we look at the `__code__` attribute for a function, it'll return a code object
    - The code object itself has attributes like:
        1. `co_varnames`
            - Parameters
            - Local variable names
        2. `co_argcount`
            - Counts the number of arguments for the function
                - **Note**: `*args` and `**kwargs` are counted as 0

In [12]:
def my_func(a, b=1, *args, **kwargs):
    i = 10
    b = min(i, b)
    return a * b

In [13]:
my_func.__code__

<code object my_func at 0x000001CF6A68D1E0, file "<ipython-input-12-1fd320731204>", line 1>

In [14]:
my_func.__code__.co_varnames

('a', 'b', 'args', 'kwargs', 'i')

- As we can see, it returned the parameters ('a', 'b', 'args', 'kwargs') and the local variable ('i')

In [15]:
my_func.__code__.co_argcount

2

- As we can see, we got two (`a` and `b`)
    - Omitted `*args` and `**kwargs`

# How can we simplify looking at these elements?

- We can use the built-in `inspect` module
    - There are many functions we can use such as:
        1. `ismethod(obj)`
        2. `isfunction(obj)`
        3. `isroutine(obj)`

# What's the difference between a function and a method?

- Let's consider an example:

In [16]:
def my_func():
    pass

In [18]:
class MyClass:
    def func(self):
        pass

In [19]:
my_obj = MyClass()

- As we can see, `my_func` and `func` are almost identical
    - The only difference is `func` takes `self` as a parameter (standard for classes) and `func` is defined inside the class
        - Therefore, `my_func` can be called independently of any instance of `MyClass`
            - However, `func` can only be called when an instance of `MyClass` exists

- Therefore:
    - `my_func` is a **function**
    - `func` is a **method**

In [20]:
import inspect

In [22]:
inspect.isfunction(my_func), inspect.ismethod(my_func)

(True, False)

In [23]:
inspect.isfunction(my_obj.func), inspect.ismethod(my_obj.func)

(False, True)

# What's a routine?

- It's a function OR a method

```python
inspect.isroutine(f) == inspect.isfunction(f) or inspect.ismethod(f)
```

In [24]:
inspect.isroutine(my_func), inspect.isroutine(my_obj.func)

(True, True)

# How can we look at the source code of our routines?

- Using `inspect.getsource`

In [26]:
print(inspect.getsource(my_func))

def my_func():
    pass



- This will return the entire definition including all documentation and annotations

- We can use `inspect.getmodule(my_func)` to see where the function was defined

In [27]:
inspect.getmodule(my_func)

<module '__main__'>

In [29]:
inspect.getmodule(inspect.getmodule)

<module 'inspect' from 'C:\\Users\\msinclair\\.pyenv\\pyenv-win\\versions\\3.7.7-amd64\\lib\\inspect.py'>

# How can we look at comments contained in our code?

- For example, consider the following lines of code:

In [30]:
#setting up variable
i = 10

# TODO: Implement function
# Some additional notes
def my_func(a, b=1):
    # Comment inside my_func
    pass

- We can use `inspect.getcomments`

In [33]:
print(inspect.getcomments(my_func))

# TODO: Implement function
# Some additional notes



- As we can see, it picked up the comments **preceding** the function definition

# What are the *signatures* of functions?

- A `Signature` instance contains an attribute called `parameters`
    - This is essentially a dictionary containing:
        - Parameter names (keys)
        - Metadata about the parameters (values)

In [41]:
def my_func(a: 'a string', 
            b: int = 1, 
            *args: 'additional positional args', 
            kw1: 'first keyword-only arg',
            kw2: 'second keyword-only arg' = 10,
            **kwargs: 'additional keyword-only args') -> str:
    """
    does something
    or other
    """
    pass

In [43]:
for param in inspect.signature(my_func).parameters.values():
    print(f'Name: {param.name}')
    print(f'Default: {param.default}')
    print(f'Annotation: {param.annotation}')
    print(f'Kind: {param.kind}')
    print('\n')

Name: a
Default: <class 'inspect._empty'>
Annotation: a string
Kind: 1


Name: b
Default: 1
Annotation: <class 'int'>
Kind: 1


Name: args
Default: <class 'inspect._empty'>
Annotation: additional positional args
Kind: 2


Name: kw1
Default: <class 'inspect._empty'>
Annotation: first keyword-only arg
Kind: 3


Name: kw2
Default: 10
Annotation: second keyword-only arg
Kind: 3


Name: kwargs
Default: <class 'inspect._empty'>
Annotation: additional keyword-only args
Kind: 4


